In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

import tensorflow as tf
import keras as keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D , Dropout,Conv3D, MaxPooling3D, Input, Deconv3D, BatchNormalization, Activation, Reshape
from keras.utils import to_categorical
from keras import optimizers

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
DATA_PATH = 'DataSet/species_1/'

In [3]:
def generator(batch_size=2):
  
  while(True):
    idx=0
    for X in glob.iglob(DATA_PATH + 'X_*.npy'):
        print(idx)
        name = X
        X = np.load(X)
        Y = np.load(name.replace('X','Y'))
        x_train = X[:30]
        Y = Y.reshape(1,64,64,64)
        y_train = np.tile(Y,(30,1,1,1))
        print(x_train.shape, y_train.shape)
        x = x_train[idx:idx+batch_size]
        y = y_train[idx:idx+batch_size]
        x = x.reshape(batch_size,256,512,1)
        y = y.reshape(batch_size,64,64,64,1)
        idx=idx+batch_size
        yield (x,y)

In [4]:
filters_in = [32,64,96]
kernel_size = [16,36,40]
strides = (4,4,4)

In [5]:
def encoder2D(kernel_size, strides):
    inputs = Input(shape=(256,512,1))

    g1 = Conv2D(filters=filters_in[0], kernel_size=kernel_size[0])(inputs)
    g1 = MaxPooling2D(pool_size=2)(g1)
    g1 = Activation(activation='relu')(g1)

    g2 = Conv2D(filters=filters_in[1], kernel_size=kernel_size[1])(g1)
    g2 = MaxPooling2D(pool_size=2)(g2)
    g2 = Activation(activation='relu')(g2)

    g3 = Conv2D(filters=filters_in[2], kernel_size=kernel_size[2])(g2)
    g3 = MaxPooling2D(pool_size=2)(g3)
    g3 = Activation(activation='relu')(g3)

#     g4 = Conv2D(filters=filters_in[3], kernel_size=kernel_size[3])(g3)
#     g4 = MaxPooling2D(pool_size=2)(g4)
#     g4 = Activation(activation='relu')(g4)

#     g5 = Conv2D(filters=filters_in[4], kernel_size=kernel_size[4])(g4)
#     g5 = MaxPooling2D(pool_size=2)(g5)
#     g5 = Activation(activation='sigmoid')(g5) 
    
    g5 = Flatten()(g3)
    
    g6 = Dense(400,activation='sigmoid')(g5)
    g6 = Activation(activation='sigmoid')(g6) 
    
    g7 = Dense(200,activation='sigmoid')(g6)
    g7 = Activation(activation='sigmoid')(g7) 
    
    g8 = Reshape((1, 1,1,200), input_shape=(200,))(g7)
    
    
    model = Model(inputs=inputs, outputs=g8)
    model.summary()

    return model 
encoder = encoder2D(kernel_size,strides)  


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 512, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 241, 497, 32)      8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 120, 248, 32)      0         
_________________________________________________________________
activation_1 (Activation)    (None, 120, 248, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 213, 64)       2654272   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 42, 106, 64)       0         
_________________________________________________________________
acti

In [6]:
kernel_size = [4,8,16,32,20]
strides = (2,2,2)
def decoder3D(kernel_size, strides):
    inputs = Input(shape=(1, 1, 1, 200))

    g1 = Deconv3D(filters=512, kernel_size=kernel_size[0],strides=strides)(inputs)
    g1 = BatchNormalization()(g1)
    g1 = Activation(activation='relu')(g1)

    g2 = Deconv3D(filters=256, kernel_size=kernel_size[1],strides=strides)(g1)
    g2 = BatchNormalization()(g2)
    g2 = Activation(activation='relu')(g2)

#     g3 = Deconv3D(filters=128, kernel_size=kernel_size[2],strides=strides)(g2)
#     g3 = BatchNormalization()(g3)
#     g3 = Activation(activation='relu')(g3)

    g4 = Deconv3D(filters=64, kernel_size=kernel_size[3])(g2)
    g4 = BatchNormalization()(g4)
    g4 = Activation(activation='relu')(g4)

    g5 = Deconv3D(filters=1, kernel_size=kernel_size[4])(g4)
    g5 = BatchNormalization()(g5)
    g5 = Activation(activation='sigmoid')(g5) 

    model = Model(inputs=inputs, outputs=g5)
    model.summary()

    return model 
decoder = decoder3D(kernel_size,strides)  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 1, 1, 200)      0         
_________________________________________________________________
conv3d_transpose_1 (Conv3DTr (None, 4, 4, 4, 512)      6554112   
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 4, 512)      2048      
_________________________________________________________________
activation_6 (Activation)    (None, 4, 4, 4, 512)      0         
_________________________________________________________________
conv3d_transpose_2 (Conv3DTr (None, 14, 14, 14, 256)   67109120  
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 14, 256)   1024      
_________________________________________________________________
activation_7 (Activation)    (None, 14, 14, 14, 256)   0         
__________

In [7]:
def create_model():
  model = Sequential()
  model.add(encoder)
  model.add(decoder)
  model.summary()
  model.compile(optimizer = 'adam', loss=['binary_crossentropy'], metrics=['mse'])
  return model

model = create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 1, 1, 1, 200)      13840792  
_________________________________________________________________
model_2 (Model)              (None, 64, 64, 64, 1)     611049541 
Total params: 624,890,333
Trainable params: 624,888,667
Non-trainable params: 1,666
_________________________________________________________________


In [ ]:
model.fit_generator(generator(4),verbose=1,steps_per_epoch=1,epochs =10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
0
(30, 256, 512) (30, 64, 64, 64)
4
(30, 256, 512) (30, 64, 64, 64)
8
(30, 256, 512) (30, 64, 64, 64)
12
(30, 256, 512) (30, 64, 64, 64)
16
(30, 256, 512) (30, 64, 64, 64)
20
(30, 256, 512) (30, 64, 64, 64)
24
(30, 256, 512) (30, 64, 64, 64)
28
(30, 256, 512) (30, 64, 64, 64)
